### Title: **Tiny Thesis** - Does injecting screw-torque make a difference ( 30 vs 80 eps) 

**Date:** 2026-02-19

**System:** Conda : act_train

**Goal:** Test 


### 1. Motivation

**Goal** : Show that ACT_FT and ACT have performance discrepancies in the plug insert task (with position randomization)

## 2. Task design and Hypothesis

### Task Description

Plug insertion is a multi-mode dexterous task. The policy must:
1. Upon seeing different tilt orientations, produce corrective motions.
2. Produce appropriate downward force to complete the insertion.

**Goal**: Show Plug-Insert performance is improved by incorporating **Screw Axis Torque**, treating it as first-class citizen.\
This should provide insights for producing "cripped" ACT baselines to test Force-Torque injection.

### Task Configurations
We perform the below tasks:
- 30 eps : 3 views + FT
- 30 eps : 3 views
- 80 eps : 3 views + FT
- 80 eps : 3 views



## 3. Task Design Considerations
**Q:** Why choose 30 / 80?\
**A:** These correspond to the dataset content in the **Plug Insert** Dataset.\
30 Episodes only contains the powerboard at a fixed location. The 80 Episode version contains positional variations in the dataset.

**Q:** Why use 3 views?\
**A:** Go with the best models.


## 4. Method

NOTE: jupyter notebooks cannot prevent process kill even with nohup. We run below in a tmux terminal.

Base ACT Training - 30 eps

In [ ]:
#!/bin/bash

python3 imitate_episodes.py \
--task_name plug_insert \
--ckpt_dir ./ckpt_dir/plug_insert \
--robot /home/khw/interbotix_ws/src/aloha/config/robot/aloha_solo \
--policy_class ACT \
--kl_weight 10 \
--chunk_size 100 \
--hidden_dim 512 \
--batch_size 16 \
--dim_feedforward 3200 \
--num_epochs 6000 \
--lr 1e-5 --seed 0

Base ACT Training - 80 eps

In [ ]:
#!/bin/bash

echo "Combining datasets..."
mv /mnt/c2d9b23a-b03e-4fdb-82ad-59f039ec9e3e/khw/plug_insert2/*.hdf5 /mnt/c2d9b23a-b03e-4fdb-82ad-59f039ec9e3e/khw/plug_insert/

sleep 30  # Increased cooldown to ensure file handles are closed and GPU is clear

In [ ]:
python3 imitate_episodes.py \
--task_name plug_insert \
--ckpt_dir ./ckpt_dir/plug_insert_base80 \
--robot /home/khw/interbotix_ws/src/aloha/config/robot/aloha_solo \
--policy_class ACT \
--kl_weight 10 \
--chunk_size 100 \
--hidden_dim 512 \
--batch_size 16 \
--dim_feedforward 3200 \
--num_epochs 10000 \
--lr 1e-5 --seed 0

#### ACT + Screw Torque training
Scripts must be run from the other repo.\
Remember to source the environment.

ACT + Screw Torque Training - 30 eps

In [ ]:
python3 imitate_episodes_screw_torque.py \
--task_name plug_insert \
--ckpt_dir ./ckpt_dir/plug_insert_screw_torque30 \
--robot /home/khw/interbotix_ws/src/aloha/config/robot/aloha_solo \
--policy_class ACT \
--kl_weight 10 \
--chunk_size 100 \
--hidden_dim 512 \
--batch_size 16 \
--dim_feedforward 3200 \
--num_epochs 6000 \
--lr 1e-5 --seed 0 \
> plug_insert_screw_torque30.log 2>&1

ACT + Screw Torque Training - 80 eps

In [ ]:
python3 imitate_episodes_screw_torque.py \
--task_name plug_insert \
--ckpt_dir ./ckpt_dir/plug_insert_screw_torque80 \
--robot /home/khw/interbotix_ws/src/aloha/config/robot/aloha_solo \
--policy_class ACT \
--kl_weight 10 \
--chunk_size 100 \
--hidden_dim 512 \
--batch_size 16 \
--dim_feedforward 3200 \
--num_epochs 10_000 \
--lr 1e-5 --seed 0

Below is the exact bash command I ran for scheduling the remaining Training.

In short, this is done by `while ps -p <PID> > /dev/null; do sleep 60; done; ./schedule_train.sh`.

The command is saved as a bash script, `schedule_train.sh`

TIP: Remember to **source the environment** in the Tmux session!!

## 5. Results + Evaluation
We can only run real world rollouts for each of the tests.\
For each training configuration, we run 20 rollouts to evaluate performance.

### Policy 1

(Directly taking previous results)

**Results:**\
Very good generalization. 3 Tries maximum and it usually gets it in.

**Failure modes:**\
At certain more extreme angles, it cannot apply enough force to complete the insertion.

### Policy 2

In [ ]:
python3 imitate_episodes_record_torque.py --task_name plug_insert --ckpt_dir ./ckpt_dir/plug_insert_base80 --robot /home/khw/interbotix_ws/src/aloha/config/robot/aloha_solo --policy_class ACT --kl_weight 10 --chunk_size 100 --hidden_dim 512 --batch_size 16 --dim_feedforward 3200 --num_epochs 10000 --lr 1e-5 --seed 0 --eval --temporal_agg

Works extremely well in distribution.\
Generalization to different positions is poor.

### Policy 3

In [ ]:
python3 imitate_episodes_screw_torque.py \
--task_name plug_insert \
--ckpt_dir ./ckpt_dir/plug_insert_screw_torque30 \
--robot /home/khw/interbotix_ws/src/aloha/config/robot/aloha_solo \
--policy_class ACT \
--kl_weight 10 \
--chunk_size 100 \
--hidden_dim 512 \
--batch_size 16 \
--dim_feedforward 3200 \
--num_epochs 6000 \
--lr 1e-5 --seed 0 --eval --temporal_agg

Results:

Succeeded once, continually adjustment.\
Fails afterwards, especially poor for some particular angles.\
Upward facing angle is best performing in terms of desired motions produced.\

Best:\
Episode 0

### Policy 4

In [ ]:
python3 imitate_episodes_screw_torque.py \
--task_name plug_insert \
--ckpt_dir ./ckpt_dir/plug_insert_screw_torque_base80 \
--robot /home/khw/interbotix_ws/src/aloha/config/robot/aloha_solo \
--policy_class ACT \
--kl_weight 10 \
--chunk_size 100 \
--hidden_dim 512 \
--batch_size 16 \
--dim_feedforward 3200 \
--num_epochs 10_000 \
--lr 1e-5 --seed 0 --eval --temporal_agg

Best:
- epsidode 1, 2

Weirdest:
- episode 4

Worse than P3:
- episode 5

Generalizaton:
- episode 6

worst:
5, 8, 9



## Design considerations 
* When policy gets stuck, it should be able to reset back to a wider view position, dynamically recompute the temporal aggregation / temporal chunk size to allow reset.
* Compliance controller might give it this tendency.

## Open Questions / Next Steps